<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical, get_file

In [2]:
url = 'https://www.gutenberg.org/files/100/100-0.txt'

doc = get_file('shakespeare.txt', url)
text = open(doc, 'rb').read().decode(encoding='utf-8-sig')
len(text)

5783552/5777367 [==============================] - 3s 0us/step


5740053

In [3]:
# Preprocessing the text 

# Cleaning
text = text.replace('\r', '')
''' Lower casing Text '''
text = text[900:-25000].lower()
''' Fixing the spacing Issue '''
text = ' '.join(text.split())


# Size
text_size = len(text)
# All the letters/characters used in the Text
vocab = sorted(set(text))

# All the letters/characters into ints 
char_to_int = {c:i for i, c in enumerate(vocab)}
int_to_char = {i:c for i, c in enumerate(vocab)}

text_integers = np.array([char_to_int[c] for c in text])

# Epoch 
seq_length = 100

X_text = []
y_text = []

for i in range(0, 100000 - seq_length,1):
    in_seq = text[i:i + seq_length]
    out_char = text[i + seq_length]
    X_text.append([char_to_int[char] for char in in_seq])
    y_text.append(char_to_int[out_char])
    
samples = len(X_text)


In [4]:
len(X_text)

99900

In [5]:
X = np.reshape(X_text, (99900, 100, 1))
X = X / len(vocab)

y = to_categorical(y_text)
y.shape

(99900, 71)

In [6]:
# Model

model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(256))
model.add(Dropout(.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 71)                18247     
Total params: 807,751
Trainable params: 807,751
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metric=['accuracy'])



history = model.fit(X, y, batch_size=1000, epochs = 50)

Train on 99900 samples
Epoch 1/50
99900/99900 [==============================] - 540s 5ms/sample - loss: 3.0580
Epoch 2/50
99900/99900 [==============================] - 513s 5ms/sample - loss: 2.9629
Epoch 3/50
99900/99900 [==============================] - 508s 5ms/sample - loss: 2.9007
Epoch 4/50
99900/99900 [==============================] - 1320s 13ms/sample - loss: 2.7901
Epoch 5/50
99900/99900 [==============================] - 515s 5ms/sample - loss: 2.7311
Epoch 6/50
99900/99900 [==============================] - 1021s 10ms/sample - loss: 2.6911
Epoch 7/50
99900/99900 [==============================] - 516s 5ms/sample - loss: 2.6611
Epoch 8/50
99900/99900 [==============================] - 515s 5ms/sample - loss: 2.6315
Epoch 9/50
99900/99900 [==============================] - 514s 5ms/sample - loss: 2.6017
Epoch 10/50
99900/99900 [==============================] - 541s 5ms/sample - loss: 2.5741
Epoch 11/50
99900/99900 [==============================] - 514s 5ms/sample - loss:

In [ ]:
# Generate Text

import textwrap

start = np.random.randint(0, len(X_text)-1)
vocab_len = len(vocab)
pattern = X_text[start]

print(f"Seed: \n {''.join([int_to_char[value] for value in pattern])}")
out = [int_to_char[value] for value in pattern]

# Generate characters
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    in_seq = [int_to_char[value] for value in pattern]
    out.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print('\n')
print("LSTM Generated Text:\n")
print(textwrap.fill(''.join(out), 80))

Seed: 
 ly steep in a cold valley-fountain of that ground: which borrowed from this holy fire of love, a dat


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN